In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scripts import  CompanyStockPriceScraper
from scripts import ADXIndex
from scipy.signal import find_peaks

## Select interesting examples

Interesting examples are those where: 
- there is a peak in the ADX index close to the day of publication,
- there is a low ADX index value over an extended period before and after the publication date. 

In [2]:
# Analysis company list 
tickers = ["CCC","CDL","EAH","HRP","IPO","LBW","LPP","MIR","MON","PRT","SFG","SNW","WTN"] # WIG_ODZIEZ
ticker = "CCC"

In [3]:
# Scrap stock value and calculate ADX index
csp = CompanyStockPriceScraper()
adx = ADXIndex()
stock_value = csp.getHistoricalPrice(ticker)
adx_index = adx.cast_to_data_frame(adx.get_index(stock_value))
stock_value = pd.concat([stock_value, adx_index], axis=1, join="inner")

In [4]:
# Get date of report publication
report_database = pd.read_csv("../database/mergedData/Annual_V.csv", index_col=[0])
report_days = report_database[report_database["Ticker"] == ticker]["Data"].to_numpy()

In [5]:
# Find inner index day
inner_index_day = np.array([np.sum(stock_value.index < report_day) for report_day in report_days])
inner_index_day = inner_index_day[inner_index_day != 0]

In [6]:
STRONG_TRENDS_RANGE = 10 # +/- days from publication date
DETECT_TRENDS_VALUE = 40 # Value ADX bigger than interpret as strong trends

In [7]:
# Check is in report publication day surroundings strong trends
# Strong trends ADX > 40 if ADX periods = 14
peak_threshold = stock_value["ADX"].mean() + stock_value["ADX"].std()

for report_day_idx in inner_index_day:
    if any(stock_value[report_day_idx-STRONG_TRENDS_RANGE:
                       report_day_idx+STRONG_TRENDS_RANGE+1]["ADX"] > DETECT_TRENDS_VALUE):
        
        # Find distance date report - ADX peak 
        peaks_idx, _ = find_peaks(stock_value["ADX"], height=DETECT_TRENDS_VALUE, distance=STRONG_TRENDS_RANGE)
        distance = peaks_idx[np.abs(report_day_idx - peaks_idx).argmin()] - report_day_idx
        print("Important")
        print(distance)
        print("-------")
    else:
        print("Not Important")
        print("-------")
        

Not Important
-------
Important
-2
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Not Important
-------
Important
9
-------
Not Important
-------
Not Important
-------
Important
6
-------


# ADX

Wskaźnik ADX (Average Directional Index) jest narzędziem analizy technicznej służącym do pomiaru siły trendu, niezależnie od jego kierunku. Nie wskazuje on, czy trend jest wzrostowy, czy spadkowy, ale pokazuje, jak silny jest obecny ruch rynkowy. ADX składa się z trzech głównych komponentów:

1. ADX – mierzy siłę trendu.
2. +DI (Positive Directional Indicator) – mierzy siłę trendu wzrostowego.
3. −DI (Negative Directional Indicator) – mierzy siłę trendu spadkowego.

Wartości te są wyrażane w przedziale od 0 do 100. Wyższe wartości ADX oznaczają silniejszy trend, natomiast niższe – słabszy.


### Interpretacja wskaźnika ADX

- ADX poniżej 20: Wskazuje na słaby lub brak trendu (rynek konsolidacyjny).
- ADX pomiędzy 20 a 40: Średnia siła trendu. Ruch rynkowy zaczyna się stawać bardziej zdecydowany.
- ADX powyżej 40: Silny trend, niezależnie od tego, czy jest to trend wzrostowy czy spadkowy.
- ADX powyżej 50: Bardzo silny trend. Rynki w takim stanie zwykle szybko się poruszają.

### Uwagi 
- Ze względu na występujące opóźnienia wskaźnik ten dobrze sprawdza się do wykrywania końca trendu.

In [195]:
# Import require libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scripts import  CompanyStockPriceScraper
from scripts import ADXIndex
from scipy.signal import find_peaks

# Prepare require instances 
csp = CompanyStockPriceScraper()
adx = ADXIndex()

# Load reports database 
reports_db = pd.read_csv("../database/mergedData/Annual_V.csv", index_col=[0])

# Configuration fields
STRONG_TRENDS_RANGE = 15 # +/- days from publication date
STRONG_TRENDS_VALUE = 40 # Value ADX bigger than interpret as strong trends
WEAK_TRENDS_VALUE   = 30 # Value ADX lower than interpret as weak trends
FRAME_OFFSET = 15

In [196]:
ticker_list = reports_db["Ticker"].unique()

n = 0
i = 0

for ticker in ticker_list:
    # Get stock value and calculate ADX
    stock_price = csp.getHistoricalPrice(ticker)
    adx_index = adx.cast_to_data_frame(adx.get_index(stock_price))
    stock_data = pd.concat([stock_price, adx_index], axis=1, join="inner")

    # Get date of report publication
    report_days = reports_db[reports_db["Ticker"] == ticker]["Data"].to_numpy()
    index_days  = np.trim_zeros([np.sum(stock_data.index < report_day) for report_day in report_days]) 

    # Analysis company 
    for index_day in index_days:
        if any(stock_data[index_day : index_day+STRONG_TRENDS_RANGE+1]["ADX"] > STRONG_TRENDS_VALUE):
            # Detect strong trends
            peak_idx =  index_day + np.argmax(stock_data[index_day : index_day+STRONG_TRENDS_RANGE+FRAME_OFFSET+1]["ADX"] > STRONG_TRENDS_VALUE)
            descent_end_idx = peak_idx + np.argmax(stock_data[peak_idx:]["ADX"] < WEAK_TRENDS_VALUE)
            increase_start_idx = peak_idx - np.argmax(stock_data[:peak_idx+1]["ADX"][::-1] < WEAK_TRENDS_VALUE) 
            if index_day - FRAME_OFFSET < increase_start_idx:
                i = i+1
            
        else:
            # No detection strong trends
            pass

        n = n+1

print(f"N={n}")
print(f"I={i}")

N=1958
I=294


In [89]:
strong_trend_offset = np.argmax(stock_data[index_day : index_day+STRONG_TRENDS_RANGE+1]["ADX"] > STRONG_TRENDS_VALUE)

In [164]:
first_peak_index = np.argmax(stock_data[index_day : index_day+STRONG_TRENDS_RANGE+1+PEAK_OFFSET]["ADX"] > STRONG_TRENDS_VALUE) + index_day
down_slope = np.argmax(stock_data[first_peak_index:]["ADX"] < 30) + first_peak_index
up_slope = first_peak_index - np.argmax(stock_data[:first_peak_index+1]["ADX"][::-1] < 30) 

In [178]:
index_day - up_slope

np.int64(21)